In [73]:
import pandas as pd
import mxnet as mx
import numpy as np

train_csv = pd.read_csv('../Titanic/train.csv', encoding='utf-8')
test_csv = pd.read_csv('../Titanic/test.csv', encoding='utf-8')

# 数据处理
def data_clean(csv):
    def name_counters(s):
        if ', Mr. ' in s:
            return ', Mr. '
        elif ', Mrs. ' in s:
            return ', Mrs. '
        elif ', Miss. ' in s:
            return ', Miss. '
        elif ', Master. ':
            return 'Ma'
        else:
            return s
    # name = test_csv['Name'].map(name_convert)
    # name.value_counts()

    def name_convert(s):
        if ', Mr. ' in s:
            return 0.25
        elif ', Mrs. ' in s:
            return 0.5
        elif ', Miss. ' in s:
            return 0.75
        elif ', Master. ':
            return 1.0
        else:
            return 0.0


    def sex_convert(s):
        if 'female' in s:
            return 0.5
        elif 'male' in s:
            return 1
        else:
            return 0

    def embarked_convert(s):
        if s!=0:
            return (ord(s)-65)/26
        else:
            return 0.0
        
        
    csv.fillna(value=0)
    pclass = csv['Pclass']/csv['Pclass'].mean()
    name = csv['Name'].map(name_convert)
    sex =  csv['Sex'].map(sex_convert)
    age = csv['Age'].fillna(value=csv['Age'].mean()/2)
    age = age/csv['Age'].mean()
    sibsp = csv['SibSp']
    parch = csv['Parch']
    fare = csv['Fare'] / csv['Fare'].mean()
    
    ticket_count = csv['Ticket'].value_counts()
    cabin_count = csv['Cabin'].value_counts()
    
    
    
    embarked = csv['Embarked'].fillna(value=0).map(embarked_convert)
    
    return pd.concat([pclass, name,sex,age,sibsp,parch,fare,embarked], axis=1)

train_data = data_clean(train_csv)
test_data = data_clean(test_csv)
train_label = train_csv.get('Survived')
test_id = test_csv.get('PassengerId')

In [74]:
# test_data['Fare']/test_data['Fare'].mean()
# test_csv

In [75]:
def mlp():
    """ 全链接网络搭建 Define the  MLP network."""
    data = mx.symbol.Variable('data')
    fc1 = mx.symbol.FullyConnected(data, name='fc1', num_hidden=128)
    relu1 = mx.symbol.Activation(fc1, name='relu1', act_type="relu")
    fc2 = mx.symbol.FullyConnected(relu1, name='fc2', num_hidden=64)
    relu2 = mx.symbol.Activation(fc2, name='relu2', act_type="relu")
    drop = mx.symbol.Dropout(data = relu2, p = 0.5)  
    fc3 = mx.symbol.FullyConnected(drop, name='fc3', num_hidden=2)
    sym = mx.symbol.SoftmaxOutput(fc3, name='softmax')
#     mx.viz.plot_network(mlp)
    print("Finish the network.")
    return sym

def get_one_layer_mlp(hidden, k,max_user,max_item):  
    # input  
    user = mx.symbol.Variable('user')  
    item = mx.symbol.Variable('item')  
    score = mx.symbol.Variable('score')  
    # user latent features  
    user = mx.symbol.Embedding(data = user, input_dim = max_user, output_dim = k,name='User_Embedding')  
    user = mx.symbol.Activation(data = user, act_type="relu")  
    #add dropout  
    user = mx.symbol.Dropout(data = user, p = 0.5)  
    user = mx.symbol.FullyConnected(data = user, num_hidden = hidden)  
    # item latent features  
    item = mx.symbol.Embedding(data = item, input_dim = max_item, output_dim = k,name='Item_Embedding')  
    item = mx.symbol.Activation(data = item, act_type="relu")  
    #add dropout  
    item = mx.symbol.Dropout(data = item, p = 0.5)  
    item = mx.symbol.FullyConnected(data = item, num_hidden = hidden)  
    # predict by the inner product  
    pred = user * item  
    pred = mx.symbol.sum_axis(data = pred, axis = 1)  
    pred = mx.symbol.Flatten(data = pred)  
    # loss layer  
    pred = mx.symbol.LinearRegressionOutput(data = pred, label = score)  
    return pred  

def cnn():
   # 网络搭建 Define the  CNN network.
    data = mx.symbol.var('data')
    # first conv layer
    conv1 = mx.symbol.Convolution(data=data, kernel=(5, 5), num_filter=20)
    tanh1 = mx.symbol.Activation(data=conv1, act_type="tanh")
    pool1 = mx.symbol.Pooling(data=tanh1, pool_type="max", kernel=(2, 2), stride=(2, 2))
    # second conv layer
    conv2 = mx.symbol.Convolution(data=pool1, kernel=(5, 5), num_filter=50)
    tanh2 = mx.symbol.Activation(data=conv2, act_type="tanh")
    pool2 = mx.symbol.Pooling(data=tanh2, pool_type="max", kernel=(2, 2), stride=(2, 2))
    # first fullc layer
    # flatten = mx.symbol.flatten(data=pool2)
    fc1 = mx.symbol.FullyConnected(data=pool2, num_hidden=500)
    tanh3 = mx.symbol.Activation(data=fc1, act_type="tanh")
    # second fullc
    fc2 = mx.symbol.FullyConnected(data=tanh3, num_hidden=2)
    # softmax loss
    cnnnet = mx.symbol.SoftmaxOutput(data=fc2, name='softmax')
    return cnnnet


In [83]:

# 数据训练分组
length = 128
train_data_group = train_data[1:(len(train_data)-length)]
valid_data = train_data[(len(train_data)-length):len(train_data)]
train_label_group =  train_label[1:(len(train_data)-length)]
valid_label = train_label[(len(train_data)-length):len(train_data)]

batch_size = 4
train_iter = mx.io.NDArrayIter(train_data.values,train_label.values,shuffle=True,
                               batch_size=batch_size,data_name='data',label_name='softmax_label')
valid_iter = mx.io.NDArrayIter(valid_data.values,valid_label.values,
                               batch_size=batch_size,data_name='data',label_name='softmax_label')

mod = mx.mod.Module(symbol=mlp(),
                    context=mx.cpu(),
                    data_names=['data'],
                    label_names=['softmax_label'])
mod.bind(data_shapes=train_iter.provide_data,
         label_shapes=train_iter.provide_label)

mod.init_params(initializer=mx.init.Uniform(scale=.2))
mod.init_optimizer(optimizer='adam', optimizer_params=(('learning_rate', 0.01),))
metric = mx.metric.create('acc')
acc = 0
for epoch in range(200):
    train_iter.reset()
    metric.reset()
    for batch in train_iter:
        mod.forward(batch, is_train=True)  # compute predictions
        mod.backward()  # compute gradients
        mod.update()  # update parameter
        mod.update_metric(metric, batch.label)
#         print(batch.data)
#         print(batch.label)
#         print(mod.get_outputs())
    train_metric = metric.get()
    metric.reset()
    mod.score(valid_iter,metric)
    valid_metric = metric.get()
    if train_metric[1] > acc:
        acc = train_metric[1]
        mod.save_params("besttrainacc")
    print('Epoch %d, Training %s Valid %s' % (epoch,train_metric, valid_metric))

Finish the network.
Epoch 0, Training ('accuracy', 0.71380471380471378) Valid ('accuracy', 0.75)
Epoch 1, Training ('accuracy', 0.7699214365881033) Valid ('accuracy', 0.8203125)
Epoch 2, Training ('accuracy', 0.78451178451178449) Valid ('accuracy', 0.8203125)
Epoch 3, Training ('accuracy', 0.78451178451178449) Valid ('accuracy', 0.8359375)
Epoch 4, Training ('accuracy', 0.80359147025813693) Valid ('accuracy', 0.8359375)
Epoch 5, Training ('accuracy', 0.79349046015712688) Valid ('accuracy', 0.8359375)
Epoch 6, Training ('accuracy', 0.80022446689113358) Valid ('accuracy', 0.859375)
Epoch 7, Training ('accuracy', 0.81144781144781142) Valid ('accuracy', 0.859375)
Epoch 8, Training ('accuracy', 0.81818181818181823) Valid ('accuracy', 0.8203125)
Epoch 9, Training ('accuracy', 0.80359147025813693) Valid ('accuracy', 0.859375)
Epoch 10, Training ('accuracy', 0.80920314253647585) Valid ('accuracy', 0.8671875)
Epoch 11, Training ('accuracy', 0.7912457912457912) Valid ('accuracy', 0.8515625)
Epoc

Epoch 100, Training ('accuracy', 0.83052749719416386) Valid ('accuracy', 0.875)
Epoch 101, Training ('accuracy', 0.82491582491582494) Valid ('accuracy', 0.859375)
Epoch 102, Training ('accuracy', 0.81705948372615034) Valid ('accuracy', 0.875)
Epoch 103, Training ('accuracy', 0.82828282828282829) Valid ('accuracy', 0.859375)
Epoch 104, Training ('accuracy', 0.8204264870931538) Valid ('accuracy', 0.8515625)
Epoch 105, Training ('accuracy', 0.82267115600448937) Valid ('accuracy', 0.84375)
Epoch 106, Training ('accuracy', 0.81818181818181823) Valid ('accuracy', 0.859375)
Epoch 107, Training ('accuracy', 0.82940516273849607) Valid ('accuracy', 0.8671875)
Epoch 108, Training ('accuracy', 0.82940516273849607) Valid ('accuracy', 0.8671875)
Epoch 109, Training ('accuracy', 0.81818181818181823) Valid ('accuracy', 0.828125)
Epoch 110, Training ('accuracy', 0.82491582491582494) Valid ('accuracy', 0.8671875)
Epoch 111, Training ('accuracy', 0.83389450056116721) Valid ('accuracy', 0.8515625)
Epoch 1

In [89]:
test_data = data_clean(test_csv)
test_iter = mx.io.NDArrayIter(test_data.values,
                               batch_size=4,data_name='data',label_name='softmax_label')
pred = mod.predict(test_iter)
pred_label = np.argmax(pred,1)

In [91]:
# Write
import csv
with open("results.csv", "w") as resFile:
    csv_writer = csv.writer(resFile)
    csv_writer.writerow(['PassengerId', 'Survived'])
    i = 1
    for i in range(0,len(test_id)):
        csv_writer.writerow([test_id[i], int(pred_label[i].asscalar())])
        i += 1

In [77]:
# int(pred_label[0].asscalar())

In [87]:
mod = mx.mod.Module(symbol=mlp(),
                    context=mx.cpu(),
                    data_names=['data'],
                    label_names=['softmax_label'])
mod.bind(data_shapes=train_iter.provide_data,
         label_shapes=train_iter.provide_label)

mod.load_params('besttrainacc')


Finish the network.


In [88]:
mod.score(valid_iter,metric)

[('accuracy', 0.8828125)]